In [1]:
from nltk.corpus import wordnet as wn

In [2]:
w1=wn.synsets('vulture')
w2=wn.synsets('stork')
w3=wn.synsets('bald')
print (w3)
print(w1[0].path_similarity(w3[0]))

[Synset('bald.v.01'), Synset('bald.s.01'), Synset('bald.s.02'), Synset('bald.s.03')]
None


In [3]:
import numpy as np
import pandas as pd
training=open("../Desktop/training.txt","r")
validation=open("../Desktop/validation.txt","r")
train_list=training.readlines()
val_list=validation.readlines()
columns=["word1","word2","feature","label"]
training_df=pd.DataFrame(columns=columns)
validation_df=pd.DataFrame(columns=columns)
words=[]
for i in range (len(val_list)):
    w=val_list[i].split(",")
    if w[2][-1]=='?':
        w[2]=w[2][:-1]
    validation_df.loc[i]=[w[0],w[1],w[2],w[3][0]] 
    words.append(w[0])
    words.append(w[1])
    words.append(w[2])
for i in range (len(train_list)):
    w=train_list[i].split(",")
    training_df.loc[i]=[w[0],w[1],w[2],w[3][0]] 
    words.append(w[0])
    words.append(w[1])
    words.append(w[2])

In [4]:
word_set=set(words)
word_dic={}
none_set=[]
for word in word_set:
    word_dic[word]=wn.synsets(word)
    if(word_dic[word]==[]):
        none_set.append(word)
none_set=set(none_set)    
none_set

{'bambi',
 'close-up',
 'crunchy',
 'lawnmowers',
 'manmade',
 'microfiber',
 'mojito',
 'non-stick',
 'portaging',
 'roading',
 'short-sighted',
 'trainable'}

In [5]:

def sim(w1,w2):
    
    if (w1 in none_set or w2 in none_set):
        return 0.05,0.05,0,0.05
    list=[]
    w1=wn.synsets(w1)
    w2=wn.synsets(w2)
    for i in w1:
        for j in w2:
            if(i.path_similarity(j)==None):
                list.append(0)
            else:
                list.append(i.path_similarity(j))
#    print (list)
    return list[0],max(list),min(list),sum(list)/len(list)

In [6]:
sim('apple','red')

(0.06666666666666667, 0.09090909090909091, 0, 0.03920325706039991)

In [7]:
def add_sim(training_df):
    fir12=[]
    fir13=[]
    fir23=[]
    max12=[]
    max23=[]
    max13=[]
    min12=[]
    min23=[]
    min13=[]
    avg12=[]
    avg23=[]
    avg13=[]
    for index,row in training_df.iterrows():
        res12=sim(row[0],row[1])
        res13=sim(row[0],row[2])
        res23=sim(row[1],row[2])
        fir12.append(res12[0])
        fir13.append(res13[0])
        fir23.append(res23[0])
        max12.append(res12[1])
        max13.append(res13[1])
        max23.append(res23[1])
        min12.append(res12[2])
        min13.append(res13[2])
        min23.append(res23[2])
        avg12.append(res12[3])
        avg13.append(res13[3])
        avg23.append(res23[3])
    training_df['fir12']=fir12
    training_df['fir13']=fir13
    training_df['fir23']=fir23
    training_df['max12']=max12
    training_df['max13']=max13
    training_df['max23']=max23
    training_df['min12']=min12
    training_df['min13']=min13
    training_df['min23']=min23
    training_df['avg12']=avg12
    training_df['avg13']=avg13
    training_df['avg23']=avg23

In [8]:
add_sim(training_df)

In [9]:
add_sim(validation_df)

In [10]:
from sklearn import svm
X=[]
Y=[]
for index, row in training_df.iterrows():
    X.append(row[4:])
    Y.append(row[3])

clf=svm.SVC()
clf.fit(X,Y)

predicted=[]
correct=0
total=0
count_1=0
count_1_right=0
count_all_1=0

for index,row in validation_df.iterrows():
    pre=clf.predict([row[4:]])  
    predicted.append(pre)
    if pre=="1":
        count_1+=1
        if pre==row[3]:
            count_1_right+=1
    if pre==row[3]:
        correct+=1
    if row[3]=="1":
        count_all_1+=1
    total+=1
validation_df["predicted"]=predicted
accuracy=correct/total
print(count_1)
print(count_1_right)
print(count_all_1)
precision=count_1_right/count_1
recall=count_1_right/count_all_1
print ("accuracy:",accuracy)
print ("precision:",precision)
print ("recall:",recall)
print ("F1:",2*precision*recall/(precision+recall))
validation_df[:100][["word1","word2","feature","label","predicted"]]



17
10
1364
accuracy: 0.5
precision: 0.5882352941176471
recall: 0.007331378299120235
F1: 0.014482259232440262


,word1,word2,feature,label,predicted
0,sandwiches,breakfast,lunch,1,[0]
1,surfboard,raft,water,0,[0]
2,banana,cucumber,long,0,[0]
3,lambs,cattle,wool,1,[0]
4,shrimp,spinach,pink,1,[0]
5,bus,taxi,passengers,0,[0]
6,garage,brick,large,1,[0]
7,walkway,alley,walk,0,[0]
8,polyurethane,polyester,material,0,[0]
9,spoon,potato,wood,1,[0]


In [11]:
validation_df

,word1,word2,feature,label,fir12,fir13,fir23,max12,max13,max23,min12,min13,min23,avg12,avg13,avg23,predicted
0,sandwiches,breakfast,lunch,1,0.166667,0.166667,0.333333,0.166667,0.166667,0.333333,0.000000,0.000000,0.000000,0.082449,0.082449,0.152998,[0]
1,surfboard,raft,water,0,0.166667,0.090909,0.100000,0.250000,0.166667,0.200000,0.000000,0.000000,0.000000,0.093847,0.084924,0.088437,[0]
2,banana,cucumber,long,0,0.142857,0.000000,0.000000,0.200000,0.000000,0.000000,0.062500,0.000000,0.000000,0.118006,0.000000,0.000000,[0]
3,lambs,cattle,wool,1,0.071429,0.100000,0.058824,0.071429,0.111111,0.058824,0.045455,0.062500,0.043478,0.059443,0.083523,0.050767,[0]
4,shrimp,spinach,pink,1,0.111111,0.076923,0.055556,0.142857,0.142857,0.125000,0.055556,0.000000,0.000000,0.083543,0.049065,0.032044,[0]
5,bus,taxi,passengers,0,0.111111,0.090909,0.071429,0.333333,0.090909,0.100000,0.000000,0.066667,0.071429,0.090122,0.084366,0.084921,[0]
6,garage,brick,large,1,0.142857,0.071429,0.083333,0.142857,0.166667,0.090909,0.071429,0.000000,0.000000,0.095310,0.057207,0.007920,[0]
7,walkway,alley,walk,0,0.142857,0.058824,0.052632,0.250000,1.000000,0.250000,0.142857,0.000000,0.000000,0.196429,0.080812,0.031669,[0]
8,polyurethane,polyester,material,0,0.250000,0.200000,0.166667,0.250000,0.200000,0.500000,0.076923,0.000000,0.000000,0.164530,0.046820,0.060653,[0]
9,spoon,potato,wood,1,0.076923,0.083333,0.100000,0.090909,0.500000,0.111111,0.058824,0.055556,0.062500,0.073023,0.088666,0.080536,[0]
